In [3]:
import numpy as np
import sys
import os
from os.path import join as pjoin


# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def mean_variance(data_dir, save_dir, joints_num): #相对表示的均值和方差
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]
    if os.path.exists(pjoin(save_dir, 'Mean.npy')):
        os.remove(pjoin(save_dir, 'Mean.npy'))
    if os.path.exists(pjoin(save_dir, 'Std.npy')):
        os.remove(pjoin(save_dir, 'Std.npy'))

    np.save(pjoin(save_dir, 'Mean.npy'), Mean)
    np.save(pjoin(save_dir, 'Std.npy'), Std)

    return Mean, Std

In [15]:
def mean_variance_test(data_dir, save_dir, joints_num): #测试集的相对表示的均值和方差
    with open('/sata/public/yyqi/Dataset/OCEAN/test.txt', 'r') as file:
        file_list = file.readlines()
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file.strip()+'.npy'))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]
    if os.path.exists(pjoin(save_dir, 'interx_mean.npy')):
        os.remove(pjoin(save_dir, 'interx_mean.npy'))
    if os.path.exists(pjoin(save_dir, 'interx_std.npy')):
        os.remove(pjoin(save_dir, 'interx_std.npy'))

    np.save(pjoin(save_dir, 'interx_mean.npy'), Mean)
    np.save(pjoin(save_dir, 'interx_std.npy'), Std)

    return Mean, Std

In [8]:
def mean_variance_raw(data_dir, save_dir): #全局位置的均值和方差
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    if os.path.exists(pjoin(save_dir, 'Mean_raw.npy')):
        os.remove(pjoin(save_dir, 'Mean_raw.npy'))
    if os.path.exists(pjoin(save_dir, 'Std_raw.npy')):
        os.remove(pjoin(save_dir, 'Std_raw.npy'))

    np.save(pjoin(save_dir, 'Mean_raw.npy'), Mean)
    np.save(pjoin(save_dir, 'Std_raw.npy'), Std)

    return Mean, Std

In [16]:
if __name__ == '__main__':
    data_dir = '/sata/public/yyqi/Dataset/OCEAN/new_joints/'
    save_dir = '/sata/public/yyqi/Dataset/OCEAN/'
    # mean, std = mean_variance(data_dir, save_dir, 22)
    mean, std = mean_variance_raw(data_dir,save_dir)

    data_dir_vec = '/sata/public/yyqi/Dataset/OCEAN/new_joint_vecs/'
    save_dir_vec = '/sata/public/yyqi/Dataset/OCEAN/'
    mean_vec, std_vec = mean_variance(data_dir_vec, save_dir_vec, 22)

    """ 计算测试集的相对表示的均值和方差 """
    data_dir_vec = '/sata/public/yyqi/Dataset/OCEAN/new_joint_vecs/'
    save_dir_vec = '/sata/public/yyqi/testOmniControl/dataset/'
    mean_vec, std_vec = mean_variance_test(data_dir_vec, save_dir_vec, 22)



    # print(mean)
    # print(std)

(156086, 263)


KeyboardInterrupt: 

### Check if your data is correct. If it's aligned with the given reference, then it is right

In [3]:
abs(mean-reference1).sum()

NameError: name 'reference1' is not defined

In [ ]:
abs(std-reference2).sum()

0.0